### Config stuff

In [1]:

from pyspark.sql import SparkSession, functions
import ConnectionConfig as cc
from pyspark.sql.functions import *
cc.setupEnvironment()

In [2]:
spark = cc.startLocalCluster("queryDB")

# MAKE TABLES AVAILABLE
This notebook is used to query the datawarehouse. Make sure your datawarehouse tables are available to query

In [3]:
# MAKE DIMENSIONS AVAILABLE AS VIEWS
spark.read.format("delta").load("spark-warehouse/dimdate").createOrReplaceTempView("dimDate")
spark.read.format("delta").load("spark-warehouse/dimsalesrep/").createOrReplaceTempView("dimSalesRep")
spark.read.format("delta").load("spark-warehouse/factsales/").createOrReplaceTempView("factSales")

In [4]:
# What is the total revenue per weekday?
spark.sql("select dd.DayOfWeek, sum(fs.revenue_mv) from factSales fs inner join dimDate dd on dd.dateSK = fs.dateSK group by DayOfWeek").show()
# What is the  revenue for each office?
spark.sql("select ds.office, sum(fs.revenue_mv) from factSales fs inner join dimSalesrep ds on ds.salesrepSK = fs.salesrepsk group by Office").show()

+---------+---------------+
|DayOfWeek|sum(revenue_mv)|
+---------+---------------+
|        6|   162008172707|
|        4|   147752834305|
|        2|   154130692053|
|        5|   151068900235|
|        1|   123479476450|
|        7|   182712450777|
|        3|   155808767203|
+---------+---------------+
+--------+---------------+
|  office|sum(revenue_mv)|
+--------+---------------+
|  Berlin|  1028779189009|
|New York|    48182104721|
+--------+---------------+
